In [ ]:
!pip install -U langchain langchain-openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install python-dotenv

In [5]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

In [6]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [11]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-92432da3-1e74-46af-b2e2-11fc197420e6-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## OutParsers

In [13]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [14]:
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

## Chaining

In [15]:
chain = model | parser

In [18]:
messages = [
    SystemMessage(content="Translate the following from English into Hindi"),
    HumanMessage(content="hi!"),
]

In [19]:
chain.invoke(messages)

'नमस्ते!'

## Creating a PromptTemplate

In [20]:
from langchain_core.prompts import ChatPromptTemplate

In [21]:
system_template = "Translate the following into {language}:"

In [22]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [24]:
result = prompt_template.invoke({"language": "Spanish", "text": "hi!"})

In [25]:
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Spanish:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [26]:
result.to_messages()

[SystemMessage(content='Translate the following into Spanish:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

## Chaining together components with LCEL

In [27]:
chain = prompt_template | model | parser

In [29]:
chain.invoke({"language": "Spanish", "text": "hi! How are you? My name is John."})

'¡Hola! ¿Cómo estás? Mi nombre es John.'

## Serving with LangServe

The serve code is in serve.py

In [33]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "Hindi", "text": "hi! How are you? My name is John."})

'नमस्ते! आप कैसे हैं? मेरा नाम जॉन है।'